In [1]:
import os
import pandas as pd
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient, models
from qdrant_client.models import VectorParams, Distance
import uuid

/Users/mhdfarhanali/Documents/Instru RAG/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Path file hasil ekstraksi PDF
data_path = "/Users/mhdfarhanali/Documents/InstruRAG/data/raw/instrumentation_sensors.txt"

with open(data_path, "r", encoding="utf-8") as f:
    raw_text = f.read()

print("Panjang teks (karakter):", len(raw_text))
print("Cuplikan awal teks:\n", raw_text[:500])

Panjang teks (karakter): 670951
Cuplikan awal teks:
 --- [HALAMAN 1] --- This E-Book and More From http://ali-almukhtar.blogspot.com --- [HALAMAN 2] --- Introduction to Instrumentation, Sensors, and Process Control --- [HALAMAN 3] --- For a listing of related titles from Artech House, turn to the back of this book --- [HALAMAN 4] --- Introduction to Instrumentation, Sensors, and Process Control William C. Dunn artechhouse.com --- [HALAMAN 5] --- Library of Congress Cataloging-in-Publication Data Dunn, William C. Introduction to instrumentation, se


In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100
)
chunks = text_splitter.split_text(raw_text)

print(f"Jumlah chunks: {len(chunks)}")
print("Contoh chunk:\n", chunks[0][:300])

Jumlah chunks: 1677
Contoh chunk:
 --- [HALAMAN 1] --- This E-Book and More From http://ali-almukhtar.blogspot.com --- [HALAMAN 2] --- Introduction to Instrumentation, Sensors, and Process Control --- [HALAMAN 3] --- For a listing of related titles from Artech House, turn to the back of this book --- [HALAMAN 4] --- Introduction to I


In [4]:
from sentence_transformers import SentenceTransformer
import numpy as np

# Load model
embedding_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# Encode + normalisasi
embeddings = embedding_model.encode(
    chunks,
    show_progress_bar=True,
    convert_to_numpy=True,
    normalize_embeddings=True
)

print("Embedding selesai!")
print(f"Jumlah chunks: {len(chunks)}")
print(f"Dimensi vektor: {embeddings.shape[1]}")


Batches: 100%|██████████| 53/53 [00:05<00:00,  9.37it/s]

Embedding selesai!
Jumlah chunks: 1677
Dimensi vektor: 384


In [5]:
# Hubungkan ke Qdrant
from qdrant_client import QdrantClient, models

qdrant = QdrantClient(host="localhost", port=6333)
collection_name = "instru_collection"

# Buat koleksi jika belum ada
try:
    qdrant.get_collection(collection_name)
    print(f"Koleksi '{collection_name}' sudah ada")
except:
    qdrant.create_collection(
        collection_name=collection_name,
        vectors_config=models.VectorParams(
            size=embeddings.shape[1],
            distance=models.Distance.COSINE
        )
    )
    print(f"Koleksi '{collection_name}' berhasil dibuat")

# Filter chunk kosong dulu
chunks = [c for c in chunks if c and c.strip()]

# Siapkan data points
points = [
    models.PointStruct(
        id=i,
        vector=embeddings[i],
        payload={"page_content": str(chunks[i])} 
    )
    for i in range(len(chunks))
]

# Upload data ke Qdrant
qdrant.upsert(
    collection_name=collection_name,
    points=points
)
print("Data berhasil diunggah ke Qdrant")

Koleksi 'instru_collection' sudah ada
Data berhasil diunggah ke Qdrant


# Interpretasi 

1. Tujuan
Notebook ini digunakan untuk mengolah teks hasil ekstraksi PDF yang sebelumnya dibuat di 0_extract_pdf.ipynb.
Tujuan utamanya adalah membagi teks yang sangat panjang menjadi potongan kecil (chunks) agar bisa dipahami dan diproses oleh model AI secara efisien.
Tahap ini penting karena model AI seperti LLM (Large Language Model) tidak bisa langsung membaca teks yang terlalu panjang, sehingga kita harus memecahnya menjadi bagian-bagian kecil tapi tetap bermakna.
Selain itu, di tahap ini juga dilakukan proses penyimpanan data hasil pemecahan ke dalam database vektor (Qdrant) agar nanti bisa dicari kembali saat chatbot menjawab pertanyaan.

2. Proses yang Dilakukan
Langkah-langkah utama di notebook ini adalah:

a. Membaca File Hasil Ekstraksi
Teks yang dihasilkan dari notebook sebelumnya (instrumentation_sensors.txt) dibuka dan dibaca sepenuhnya.
Dari hasil pembacaan, diketahui panjang teks mencapai 670.951 karakter — ini menandakan bahwa isi buku sudah lengkap dan berhasil terbaca dengan baik.
Cuplikan awal teks yang terlihat berisi bagian pembuka dari buku seperti halaman judul, informasi penerbit, dan keterangan katalog.

b. Memecah Teks Menjadi Potongan (Chunks)
Langkah berikutnya adalah membagi teks panjang tersebut menjadi potongan-potongan kecil agar lebih mudah dikelola.
Untuk itu digunakan RecursiveCharacterTextSplitter dari library langchain_text_splitters.
Setiap potongan memiliki ukuran sekitar 500 karakter, dan setiap potongan saling tumpang tindih sedikit sebanyak 100 karakter.
Overlap ini berguna agar tidak ada konteks penting yang terpotong di antara dua potongan teks.
Setelah proses pemecahan selesai, didapatkan hasil:
Jumlah total chunks: 1677
Contoh isi chunk pertama: bagian pembuka dari buku (judul, pengantar, dan informasi awal).

c. Menyimpan ke Database Qdrant
Setelah semua chunks terbentuk, langkah berikutnya adalah menyimpan data hasil potongan ini ke dalam database vektor (Qdrant).
Qdrant digunakan karena mampu menyimpan data dalam bentuk vektor yang bisa dicari dengan metode similarity search.
Artinya, ketika nanti chatbot menerima pertanyaan, sistem akan mencari potongan teks yang paling relevan di database Qdrant.
Pesan di terminal menunjukkan bahwa koleksi (collection) bernama "instru_collection" sudah ada, sehingga data baru bisa langsung ditambahkan tanpa perlu membuat ulang koleksi.
Hasil akhirnya muncul pesan:
Koleksi 'instru_collection' sudah ada
Data berhasil diunggah ke Qdrant
Ini berarti data sudah tersimpan dengan baik di database dan siap digunakan untuk tahap berikutnya, yaitu indexing dan retrieval.

3. Hasil Akhir
Panjang teks mentah: 670.951 karakter
Total chunks yang dihasilkan: 1677 potongan teks
Database vektor (Qdrant) berhasil diisi dengan semua potongan teks
Tidak ada error selama proses
Secara keseluruhan, proses berjalan sangat lancar dan hasilnya sesuai harapan.

4. Kesimpulan
Tahap 1_ingestion.ipynb berhasil mempersiapkan data agar bisa digunakan dalam sistem RAG.
Sekarang data buku sudah dipecah menjadi ribuan potongan kecil yang bisa dicari secara cepat dan akurat oleh model AI.
Dengan langkah ini, chatbot Instru RAG nanti bisa menjawab pertanyaan berdasarkan isi buku karena semua informasi teknis sudah tersimpan dalam bentuk vektor di Qdrant.

## Catatan Pribadi
Menurut saya, tahap ini seperti memotong buku menjadi potongan-potongan kecil supaya bisa “dimakan” oleh AI.
Awalnya saya agak bingung kenapa teks harus dipecah-pecah, tapi setelah dijelaskan, ternyata ini penting karena model AI punya batas panjang input.
Bagian paling menarik adalah saat data diunggah ke Qdrant dan muncul pesan “Data berhasil diunggah ke Qdrant”.
Itu artinya sistem saya mulai “punya memori”, dan dari sinilah chatbot bisa mulai belajar menjawab dari isi buku teknik asli.